# Contact-High-School

In [1]:
using Pkg
Pkg.activate("/work/users/s/h/shiyi/hypergraph_with_curvature/orchid")
using Orchid, SparseArrays, DelimitedFiles, Base.Threads, JSON

  Activating project at `/work/users/s/h/shiyi/hypergraph_with_curvature/orchid`
[ Info: Precompiling Orchid [65b37873-18c8-4f67-ad9c-7e8325cd4959]


In [15]:
# Function to parse the edge list
parse_edgelist(fp) = [parse.(Int, split(r, '\t')) for r in readlines(fp) if r != ""]

# Function to generate incidence matrix
function generate_incidence_matrix(H_list::Vector{Vector{Int64}})
    all_nodes = Set{Int}()
    for edge in H_list
        for node in edge
            push!(all_nodes, node)
        end
    end
    nodes = sort(collect(all_nodes))
    num_edges = length(H_list)
    num_nodes = length(nodes)
    incidence_matrix = zeros(Int, num_edges, num_nodes)
    for (i, edge) in enumerate(H_list)
        for node in edge
            col_index = findfirst(x -> x == node, nodes)
            if col_index !== nothing
                incidence_matrix[i, col_index] = 1
            end
        end
    end
    return sparse(incidence_matrix)
end

function compute_horc(file_path)
    println("Start $file_path")
    H_list = parse_edgelist(file_path)
    H_Inc = generate_incidence_matrix(H_list)
    horc = hypergraph_curvatures(
        Orchid.DisperseWeightedClique,
        Orchid.AggregateMean,
        H_Inc,
        0.0,
        Orchid.CostOndemand
    )
    return horc
end

function process_and_save(file_path::String, output_json::String)
    horc = compute_horc(file_path)
    
    open(output_json, "w") do io
        JSON.print(io, horc)
    end
end


process_and_save (generic function with 1 method)

In [12]:
file_path = "./derived_data/hyperedges.tsv"
output_json = "./derived_data/horc.json"

"./derived_data/horc.json"

## computation time

In [18]:
@time horc = compute_horc(file_path);

Start ./derived_data/hyperedges.tsv


[ Info: Preparing Input
[ Info: Preparing Neighborhoods
[ Info: Preparing Ondemand Cost Computation
[ Info: Preparing Dispersion
[ Info: Computing Dispersions
[ Info: Computing Directional Curvature


  3.820983 seconds (905.86 k allocations: 1.383 GiB, 16.77% gc time)


[ Info: Computing Node Curvature Neighborhood
[ Info: Computing Edge Curvature
[ Info: Computing Node Curvature Edges


## save

In [16]:
process_and_save(file_path,output_json)

Start ./derived_data/hyperedges.tsv


[ Info: Preparing Input
[ Info: Preparing Neighborhoods
[ Info: Preparing Ondemand Cost Computation
[ Info: Preparing Dispersion
[ Info: Computing Dispersions
[ Info: Computing Directional Curvature
[ Info: Computing Node Curvature Neighborhood
[ Info: Computing Edge Curvature
[ Info: Computing Node Curvature Edges
